In [19]:
from pynput import mouse
import os
import settings
import cv2
import time
import pyautogui
import numpy as np
import datetime
import pytesseract
import glob
from PIL import Image
# import ctypes
# PROCESS_PER_MONITOR_DPI_AWARE = 2
# ctypes.windll.shcore.SetProcessDpiAwareness(PROCESS_PER_MONITOR_DPI_AWARE)

left_up_x, left_up_y, right_down_x, right_down_y = [0,0,0,0]
num_press = 0

def main():
    # スクリーンショットの範囲を決定する
    left, up, width, height = decide_region()
    # スクリーンショット保存用ディレクトリを作成
    os.makedirs("Screenshot", exist_ok=True)
    # テンプレートマッチング用の画像作る
    gameset_gray, gameset_mask = make_template("Image/gameset_gray.png", "Image/gameset_mask.png")
    # スタート時間
    st = time.time()
    # 今の1Pのキャラクター名
    char_name_p1 = ""
    # 今の2Pののキャラクター名
    char_name_p2 = ""
    
    # 範囲の指定が終わったらスクリーンショットを撮る
    while True:
        try:
            time.sleep(0.2)
#             print(time.time()-st)
#             st = time.time()
            # 指定した範囲でスクリーンショットを撮影
            im = pyautogui.screenshot(region=(left, up, width, height))
#             im = Image.open(f"Screenshot/230110092419_start.png")
            # OpenCVで扱えるようにImage型からndarrayに変換して，RGBをBGRにする
            im = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)
            # スクリーンショットをとった時間を記録
            ss_time = datetime.datetime.now().strftime("%y%m%d%H%M%S")
            path = "Screenshot/" + ss_time
            # 画像をリサイズ
            im_resize = cv2.resize(im, dsize=(settings.DSIZE_W, settings.DSIZE_H))
            # 撮影したスクリーンショットをグレースケールに変換
            im_gray = cv2.cvtColor(im_resize, cv2.COLOR_BGR2GRAY)
            
            # このフレームが対戦終了タイミングかどうか
            if is_finish(im_gray, gameset_gray, gameset_mask):
                # 火花が消えるのを待って撮り直す
                time.sleep(0.3)
                im = pyautogui.screenshot(region=(left, up, width, height))
                im = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)
                im_resize = cv2.resize(im, dsize=(settings.DSIZE_W, settings.DSIZE_H))
                # スクリーンショットを保存
                cv2.imwrite(f"{path}_finish.png", im_resize)
                print("対戦終了")
                # 勝者を判定する
                winner = judge_winner(im_resize)
                if winner == 0:
                    print("勝者不明")
                else:
                    print(f"勝者はPlayer{winner}")
                # ファイルに対戦結果を保存
                with open("battleLog.txt", mode="a") as f:
                    win_lose_str = ""
                    if winner == 1:
                        win_lose_str = "WIN-LOSE"
                    elif winner == 2:
                        win_lose_str = "LOSE-WIN"
                    else:
                        win_lose_str = "FAILED_TO_READ"
                    if char_name_p1 == "":
                        char_name_p1 = "FAILED_TO_READ"
                    if char_name_p2 == "":
                        char_name_p2 = "FAILED_TO_READ"
                    f.write(f"{char_name_p1},{win_lose_str},{char_name_p2}\n")
                    char_name_p1 = ""
                    char_name_p2 = ""
                # 5秒ストップして次のスクリーンショットへ
                time.sleep(5)
                continue
            
            # Canny法でエッジを検出
            im_edges = cv2.Canny(im_gray, settings.CANNY_THRESHOLD1, settings.CANNY_THRESHOLD2, L2gradient=True)
            
            # Hough変換で直線を検出
            im_lines = cv2.HoughLines(im_edges, rho=1, theta=np.pi/180, threshold=settings.HOUGH_THRESHOLD)
            
            # このフレームが対戦開始タイミングかどうか
            if is_start(im_lines):
                # スクリーンショットを保存
                # 画像を保存するパスは "Screenshot/年月日時分秒.png"
                cv2.imwrite(f"{path}_start.png", im_resize)
                print("対戦開始")
                # キャラ名が登録されてない(対戦がまだ始まってない)ならキャラ名登録
                if char_name_p1 == "" and char_name_p2 == "":
                    char_name_p1, char_name_p2 = read_char_name(im_resize)
                # キャラ名登録されてたら対戦終了の検知に失敗してるからファイルに保存してからキャラ名上書き
                else:
                    with open("battleLog.txt", mode="a") as f:
                        if char_name_p1 == "":
                            char_name_p1 = "FAILED_TO_READ"
                        if char_name_p2 == "":
                            char_name_p2 = "FAILED_TO_READ"
                        f.write(f"{char_name_p1},FAILED_TO_READ,{char_name_p2}\n")
                        char_name_p1, char_name_p2 = read_char_name(im_resize)
                # 5秒ストップして次のスクリーンショット
                time.sleep(5)
                continue
            # 対戦開始でも終了でもない画像を保存(確認用)
#             cv2.imwrite(f"{path}.png", im_resize)
            # 実験中は繰り返し一回で終わり
#             break
        # プログラムが停止されたら終わり
        except KeyboardInterrupt:
            break
    for filename in  glob.glob("screenshot*"):
        os.remove(filename)
    print("end")
    
# スクリーンショット範囲の左上と右下をクリックで決定する
def decide_region():
    # マウスのクリックを検知(クリック2回目が左上，3回目が右下でスクショ範囲を指定)
    with mouse.Listener(on_click=on_click) as listener:
        listener.join()
    # スクリーンショット範囲の計算(2倍したらいい理由は謎)
    left = left_up_x*2
    up = left_up_y*2
    width = (right_down_x-left_up_x)*2
    height = (right_down_y-left_up_y)*2
    return left, up, width, height

# マウスをクリックしたときの処理
def on_click(x, y, button, pressed):
    global left_up_x, left_up_y, right_down_x, right_down_y, num_press
    if pressed:
        # 1回目のクリックはウィンドウの切り替え用で何もしない
        if num_press == 0:
            num_press = 1
        # 2回目のクリックはスクリーンショットを撮る範囲の左上を指定
        elif num_press == 1:
            num_press = 2
            left_up_x, left_up_y = x, y
            return (x, y)
        # 3回目のクリックはスクリーンショットを撮る範囲の右下を指定して終了
        else:
            right_down_x, right_down_y = x, y
            return False

# テンプレートマッチング用の画像(テンプレートとマスク)を作る
def make_template(template_path, mask_path):
    # リサイズの縦横の長さ
    DSIZE_W = settings.DSIZE_W
    DSIZE_H = settings.DSIZE_H
    # 引数で与えられたパスからグレースケールで画像を読み込んでリサイズ
    template_gray = cv2.resize(cv2.imread(template_path, cv2.IMREAD_GRAYSCALE), dsize=(DSIZE_W, DSIZE_H))
    template_mask = cv2.resize(cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE), dsize=(DSIZE_W, DSIZE_H))
    # 128を閾値としてマスク画像を二値化
    _, template_mask = cv2.threshold(template_mask, 128, 255, cv2.THRESH_BINARY)
    # 画像の周り1/10省いてちょっとずれてもマッチングできるようにする
    template_gray = template_gray[int(DSIZE_H/20):int(-DSIZE_H/20), int(DSIZE_W/20):int(-DSIZE_W/20)]
    template_mask = template_mask[int(DSIZE_H/20):int(-DSIZE_H/20), int(DSIZE_W/20):int(-DSIZE_W/20)]
    return template_gray, template_mask

# 対戦終了のタイミングならTrue
def is_finish(im_gray, gameset_gray, gameset_mask):
    res = cv2.matchTemplate(im_gray, gameset_gray, cv2.TM_CCOEFF_NORMED, gameset_mask)
    _, max_val, _, _ = cv2.minMaxLoc(res)
    is_fin = True if max_val > 0.37 else False
#     if is_fin:
#         print(f"max_val: {max_val}")
    return is_fin

# 対戦開始のタイミングならTrue
def is_start(im_lines, error=10**-2):
    # 直線が検出されたときだけ，このフレームが対戦開始タイミングかどうかを判定する
    if im_lines is not None:
        # いい感じの直線の数
        line_count = 0
        # 画面の上部かどうかを判断する閾値
        threshold = settings.DSIZE_H*0.2
        for line in im_lines:
            line = line[0]
            # 直線が画面の上部かどうか(上部でないなら次の線)
            if line[0] > threshold:
                continue
            # いい感じの角度の直線ならカウントする
            if np.abs(line[1] - settings.STARTING_RADIAN) < error:
                line_count += 1
                # 2本あれば対戦開始のタイミング
                if line_count == 2:
                    return True
    return False

# 勝者を返す(1: 1Pの勝ち, 2: 2Pの勝ち, 0: 勝者不明)
def judge_winner(im_resize):
    # リサイズの縦横のサイズ
    DSIZE_H = settings.DSIZE_H
    DSIZE_W = settings.DSIZE_W
    
    # 1P,2Pの蓄積パーセントの範囲をスライスして赤色だけ抽出する
    red_p1 = im_resize[int(DSIZE_H*0.844):int(DSIZE_H*0.932), int(DSIZE_W*0.252):int(DSIZE_W*0.252)+int(DSIZE_W*0.105), 2]
    red_p2 = im_resize[int(DSIZE_H*0.844):int(DSIZE_H*0.932), int(DSIZE_W*0.638):int(DSIZE_W*0.638)+int(DSIZE_W*0.105), 2]

    # 二値化
    _, nichika_p1 = cv2.threshold(red_p1, 140, 255, cv2.THRESH_BINARY)
    _, nichika_p2 = cv2.threshold(red_p2, 140, 255, cv2.THRESH_BINARY)
#     cv2.imwrite("nichika_p1.png", nichika_p1)
#     cv2.imwrite("nichika_p2.png", nichika_p2)

    # 白黒反転して外側から黒を白に塗りつぶす(背景を白にする)
    rev_p1 = 255 - nichika_p1
    rev_p2 = 255 - nichika_p2
    # 黒で1周囲んで背景を塗りつぶしやすくする
    zero_w = np.zeros((1, len(rev_p1[0])), dtype=np.uint8)
    rev_p1 = np.concatenate([zero_w, rev_p1, zero_w], axis=0)
    rev_p2 = np.concatenate([zero_w, rev_p2, zero_w], axis=0)
    zero_h = np.zeros((len(rev_p1), 1), dtype=np.uint8)
    rev_p1 = np.concatenate([zero_h, rev_p1, zero_h], axis=1)
    rev_p2 = np.concatenate([zero_h, rev_p2, zero_h], axis=1)
    _, fill_p1, _, _ = cv2.floodFill(rev_p1, mask=None, seedPoint=(0, 0), newVal=255)
    _, fill_p2, _, _ = cv2.floodFill(rev_p2, mask=None, seedPoint=(0, 0), newVal=255)
#     cv2.imwrite("fill_p1.png", fill_p1)
#     cv2.imwrite("fill_p2.png", fill_p2)

    # 黒いノイズを除去(オープニング)
    # カーネルはこんな形
    # [[0,0,1,0,0],
    #  [0,1,1,1,0],
    #  [1,1,1,1,1],
    #  [0,1,1,1,0],
    #  [0,0,1,0,0]]
    kernel = np.array([[0,0,1,0,0], [0,1,1,1,0], [1,1,1,1,1], [0,1,1,1,0], [0,0,1,0,0]], np.uint8)
    closing_p1 = cv2.morphologyEx(fill_p1, cv2.MORPH_CLOSE, kernel)
    closing_p2 = cv2.morphologyEx(fill_p2, cv2.MORPH_CLOSE, kernel)
#     cv2.imwrite("closing_p1.png", closing_p1)
#     cv2.imwrite("closing_p2.png", closing_p2)
    
    # 文字認識結果
    im_str_1 = pytesseract.image_to_string(closing_p1, config="--psm 7").replace("\n", " ")[:-1]
    im_str_2 = pytesseract.image_to_string(closing_p2, config="--psm 7").replace("\n", " ")[:-1]
    print(f"player1:{im_str_1}")
    print(f"player2:{im_str_2}")
    
    # 蓄積パーセントを見て勝者判定
    winner = 0
    # 数字の判定に使う文字列
    NUM_STRING = "1234567890"
    for num_ch in im_str_1:
        if num_ch in NUM_STRING:
            winner += 1
            break
    for num_ch in im_str_2:
        if num_ch in NUM_STRING:
            winner += 2
            winner %= 3
            break
    return winner

# 対戦開始タイミングの画像から1P,2Pのキャラクターの名前をとってきて文字列で返す．
def read_char_name(im_resize):
    # リサイズの縦横のサイズ
    DSIZE_H = settings.DSIZE_H
    DSIZE_W = settings.DSIZE_W
    
    # 1P,2Pのキャラクター名の範囲をスライス，1Pは青色，2Pは赤色だけ抽出する
    im_p1 = im_resize[:int(DSIZE_H*0.14), int(DSIZE_W*0.074):int(DSIZE_W*0.074)+int(DSIZE_W*0.37), 0]
    im_p2 = im_resize[:int(DSIZE_H*0.14), int(DSIZE_W*0.574):int(DSIZE_W*0.574)+int(DSIZE_W*0.37), 2]
    cv2.imwrite("2p.png", im_p2)
    # 二値化
    _, im_p1 = cv2.threshold(im_p1, 230, 255, cv2.THRESH_BINARY)
    _, im_p2 = cv2.threshold(im_p2, 230, 255, cv2.THRESH_BINARY)
    
    # キャラクター名をtesseractで読み取る
    char_name_p1 = pytesseract.image_to_string(im_p1).replace("\n", " ")[:-1]
    char_name_p2 = pytesseract.image_to_string(im_p2).replace("\n", " ")[:-1]
    # キャラクター名が1行のときは読み取れないことが多いから，読み取れてなかったら画像全体を1行として読み取るオプションつける
    if char_name_p1 == "":
        char_name_p1 = pytesseract.image_to_string(im_p1, config="--psm 7").replace("\n", " ")[:-1]
    if char_name_p2 == "":
        char_name_p2 = pytesseract.image_to_string(im_p2, config="--psm 7").replace("\n", " ")[:-1]
    print(f"char_name_p1: {char_name_p1}")
    print(f"char_name_p2: {char_name_p2}")
    return char_name_p1, char_name_p2

# Hough変換で求めたθとρをもとに直線を引く
def draw_line(im, theta, rho):
    h, w = im.shape[:2]
    # 直線が垂直のとき
    if np.isclose(np.sin(theta), 0):
        x1, y1 = rho, 0
        x2, y2 = rho, h
    # 直線が垂直じゃないとき
    else:
        # 直線の式を式変形すればcalc_yの式がもとまる．
        calc_y = lambda x: rho / np.sin(theta) - x * np.cos(theta) / np.sin(theta)
        x1, y1 = 0, calc_y(0)
        x2, y2 = w, calc_y(w)
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    # 直線を描画 (imにアルファチャンネルがあると黒い線分しか引かれなくなる)
    cv2.line(im, (x1, y1), (x2, y2), (0, 255, 0), 2)

if __name__ == "__main__":
    main()

対戦開始
char_name_p1: KEN
char_name_p2: MARIO *
end
